### HyperLogLog v3

In [1]:
sc.stop()

In [3]:
# download_file 
#! beeline -u "jdbc:hive2://ds-hadoop-cs01p:10000/" -n kposminin --incremental=true --showheader=false --outputformat=tsv2 
# --maxwidth=5000 --silent=true --showWarnings=false --showNestedErrs=false --verbose=false --nullemptystring=true 
#-e "select phone_mobile, approve, call_ymd, urlfr, ymd from user_kposminin.ccall_visits order by phone_mobile" > ccall_cred_visits.txt
#! head -1 /data1/share/kosm/data/ccall_cred_visits.txt


89000010119	0	2016-12-12	vk.com#id48424537	2016-05-09


In [22]:
from HLL import HyperLogLog as Hll

hll_len = 14

i = 0
prev_phone_mobile = None
prev_approve = None
buf = {} #Hll(hll_len)
with open('/data1/share/kosm/data/ccall_cred_visits.txt','r') as f:
    with open('/data1/share/kosm/data/ccall_cred_hll_train.tsv', 'w') as fo:
        with open('/data1/share/kosm/data/ccall_cred_hll_test.tsv', 'w') as fot:
            for line in f:
                (phone_mobile, approve, call_ymd, urlfr, ymd) = line.strip().split('\t')
                key = '%s_%s_%s' % (phone_mobile, approve, call_ymd)
                if phone_mobile != prev_phone_mobile:
                    if prev_phone_mobile is not None:
                        for key in buf:
                            if key.split('_')[2] < '2016-12-01':
                                fo.write('%s\t%s\n' % (key.split('_')[1], '\t'.join([str(e) for e in buf[key].registers()])))
                            else:
                                fot.write('%s\t%s\n' % (key.split('_')[1], '\t'.join([str(e) for e in buf[key].registers()])))
                    prev_phone_mobile = phone_mobile
                    buf = {}
                    i += 1
                    if (i % 10000 == 0):
                        print(i)
                if(ymd < call_ymd):
                    if not key in buf:
                        buf[key] = Hll(hll_len)
                    buf[key].add(urlfr)           
            

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000


In [24]:
!shuf /data1/share/kosm/data/ccall_cred_hll_test.tsv > /data1/share/kosm/data/ccall_cred_hll_test_shuf.tsv

In [25]:
!shuf /data1/share/kosm/data/ccall_cred_hll_train.tsv > /data1/share/kosm/data/ccall_cred_hll_train_shuf.tsv

In [27]:
! wc -l /data1/share/kosm/data/ccall_cred_hll_*_shuf.tsv

      58260 /data1/share/kosm/data/ccall_cred_hll_test_shuf.tsv
     648888 /data1/share/kosm/data/ccall_cred_hll_train_shuf.tsv
     707148 total


In [ ]:
!time /home/a.melnikov/tmp/LightGBM-master/lightgbm config=/home/a.melnikov/projects/web-scoring/train-hll-tri.conf

In [37]:
! cat /home/k.osminin/lgbm_files/train.conf > /home/k.osminin/lgbm_files/cred_log_20170322_1.txt
! nohup /data1/share/LightGBM/lightgbm config=/home/k.osminin/lgbm_files/train.conf data=/data1/share/kosm/data/ccall_cred_hll_train_shuf.tsv valid=/data1/share/kosm/data/ccall_cred_hll_test_shuf.tsv >> /home/k.osminin/lgbm_files/cred_log_20170322_1.txt

^C


In [36]:
! -tail -1000 /home/k.osminin/lgbm_files/cred_log_20170321.txt

[LightGBM] [Info] Iteration: 440, /data1/share/kosm/data/ccall_cred_hll_test_shuf.tsv's : log loss : 0.658835
[LightGBM] [Info] Iteration: 440, /data1/share/kosm/data/ccall_cred_hll_test_shuf.tsv's : AUC : 0.631768
[LightGBM] [Info] 8512.872541 seconds elapsed, finished iteration 440
[LightGBM] [Info] Iteration: 441, training's : log loss : 0.606033
[LightGBM] [Info] Iteration: 441, training's : AUC : 0.750734
[LightGBM] [Info] Iteration: 441, /data1/share/kosm/data/ccall_cred_hll_test_shuf.tsv's : log loss : 0.658859
[LightGBM] [Info] Iteration: 441, /data1/share/kosm/data/ccall_cred_hll_test_shuf.tsv's : AUC : 0.631713
[LightGBM] [Info] 8531.969935 seconds elapsed, finished iteration 441
[LightGBM] [Info] Iteration: 442, training's : log loss : 0.605931
[LightGBM] [Info] Iteration: 442, training's : AUC : 0.750917
[LightGBM] [Info] Iteration: 442, /data1/share/kosm/data/ccall_cred_hll_test_shuf.tsv's : log loss : 0.658866
[LightGBM] [Info] Iteration: 442, /data1/share/kosm/data/ccall

#### Лучший AUC ROC 0.632655. Iteration: 336

In [44]:
! cat /home/k.osminin/lgbm_files/train.conf > /home/k.osminin/lgbm_files/cred_log_20170322_1.txt
! nohup /data1/share/LightGBM/lightgbm config=/home/k.osminin/lgbm_files/train.conf data=/data1/share/kosm/data/ccall_cred_hll_train_shuf.tsv valid=/data1/share/kosm/data/ccall_cred_hll_test_shuf.tsv >> /home/k.osminin/lgbm_files/cred_log_20170322_1.txt &2>1

nohup: redirecting stderr to stdout


In [55]:
! tail -3300 /home/k.osminin/lgbm_files/cred_log_20170322_1.txt


        # task type, support train and predict
        task = train
        
        # boosting type, support gbdt for now, alias: boosting, boost
        boosting_type = gbdt
        
        # application type, support following application
        # regression , regression task
        # binary , binary classification task
        # lambdarank , lambdarank task
        # alias: application, app
        objective = binary

        # eval metrics, support multi metric, delimite by ',' , support following metrics
        # l1 
        # l2 , default metric for regression
        # ndcg , default metric for lambdarank
        # auc 
        # binary_logloss , default metric for binary
        # binary_error
        metric = auc,binary_logloss
        
        # frequence for metric output
        metric_freq = 1
        
        # true if need output metric for training data, alias: tranining_metric, train_metric
        is_training_metric = true
        
        # number of bins for f

648888 /data1/share/kosm/data/ccall_cred_hll_train_shuf.tsv
